In [2]:
import json
import os

with open("..\Material\itn\solent_itn.json") as json_file:
    data = json.load(json_file)
    print(len(data))
    print(type(data))  
    

4
<class 'dict'>


In [9]:
data.keys()

dict_keys(['roadlinks', 'roadnodes', 'road', 'routeinfo'])